# Scoring with RAG

In [ ]:
import os
import ocrmypdf
from PyPDF2 import PdfReader
import re
import json
import fitz  # PyMuPDF for PDF extraction
import chromadb  # Vector Database
from tqdm import tqdm
import logging
import requests
import time
import pandas as pd
import numpy as np
import torch 
from torch import nn
from torch.optim import AdamW  
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd 
from io import BytesIO
from transformers import AutoModelForSequenceClassification, AutoModelForCausalLM, AutoTokenizer, pipeline, BertTokenizer, BertModel, Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from spacy import displacy
### if u got strong gpu w cuda, should change to gpu, average laptop cpu takes too long *cough* mac book users
torch.set_default_device("cpu")

if torch.cuda.is_available():
    torch.set_default_device("cuda")
    print("running on cuda")

import random
import json
import google.generativeai as genai

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from googlesearch import search
from fuzzywuzzy import fuzz 
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
import sys


running on cuda


# RAG testing ignore for move into py

In [8]:
df = pd.read_csv("../files/labeled_pdfs_1003.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../files/labeled_pdfs_1003.csv'

Adding into DB -> shift into PY

In [16]:


def company_year_exists(company, year):
    """
    Check if a document with the given company and year already exists in the collection.
    """
    results = collection.query(
        query_texts=[""],  # Empty query text since we're filtering solely by metadata
        n_results=1,
        where={
    "$and": [
        {"company": company},
        {"year": float(year)}
    ]

}

    )
    # Check if any document was returned
    if results.get("documents") and results["documents"][0]:
        return True
    return False

def add_documents_from_csv(file_path):
    client = chromadb.PersistentClient(path="../chromadb_1003")  # Stores DB in ./chroma_db
    collection = client.get_or_create_collection(name="dsa3101")
    """
    Add documents from a CSV file to the collection.
    Assumes CSV has columns: 'esg_text', 'company', 'year', 'industry'.
    """
    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        logging.error(f"Failed to load {file_path}: {e}")
        return

    # Capture the starting document count to keep doc_ids consistent
    groups = df.groupby(["company", "year"])

    # Process each group only once
    for (company, year), group_df in tqdm(groups, total=len(groups), desc="Processing groups", unit="group", ncols=100):
        if company_year_exists(company, year):
            print(f"Group for {company} ({year}) already exists. Skipping all documents for this group.")
            continue

        # Capture starting count for unique doc_ids for this group
        starting_count = collection.count()
        # Add all rows in the group
        for i, (_, row) in enumerate(group_df.iterrows()):
            doc_text = row["esg_text"]
            doc_company = row["company"]
            doc_year = row["year"]
            doc_industry = row["industry"]
            doc_country = row["country"]

            doc_id = f"doc_{starting_count + i}"
            print(f"Adding document {doc_id} for {doc_company} ({doc_year})")
            collection.add(
                ids=[doc_id],
                documents=[doc_text],
                metadatas=[{
                    "company": doc_company, 
                    "year": doc_year, 
                    "industry": doc_industry,
                    "country": doc_country
                }]
            )

add_documents_from_csv("../files/labeled_pdfs_1003.csv")



ERROR:root:Failed to load ../files/labeled_pdfs_1003.csv: [Errno 2] No such file or directory: '../files/labeled_pdfs_1003.csv'


In [29]:
client = chromadb.PersistentClient(path="../chromadb_1003")  # Stores DB in ./chroma_db

collection = client.get_or_create_collection(name="dsa3101")

print(collection.count())  

71212


Ignore this below, its just to check if its working

In [31]:
print(company_year_exists("UOBGROUP",2023.0))
client = chromadb.PersistentClient(path="../chromadb_1003")  # Stores DB in ./chroma_db
collection = client.get_collection(name="dsa3101")

with open("../../files/scoring_queries/sg_bank_query.json", "r") as file:
    esg_metrics = json.load(file)

metric_name = esg_metrics[0]
print(metric_name)  
query = esg_metrics[0]['Total Greenhouse Gas Emissions']['value_query']
print(query)

def retrieve_esg_text(company, query):

    results = collection.query(query_texts=[query], n_results=15,where={"$and": [{"company": company[0]}, {"year": company[1]}]})

    if not results.get("documents") or not results["documents"][0]:
        print(f"Company '{company[0]}' of '{company[1]}' is not in database, stopping code and throwing error...")
        sys.exit(1)
    return results

company_info =   ("UOBGROUP", 2023.0)

print(retrieve_esg_text(company_info, query))


True
{'Total Greenhouse Gas Emissions': {'value_query': "Extract the total carbon emissions in tcO2e for the year 2023. This may be presented as 'Total Scope 1 and 2 Emissions'. Please output the extracted values in the specified structure seen in the key-value pair for extracted_values below and try not to leave any blanks. Write a brief description that summarizes the extracted values.", 'extracted_values': {'total_emissions': '', 'description': ''}, 'scoring_query': 'Based on the values extracted from value_query, give a score based on this criteria: 1 - (total_emissions/100000). Return the final_score as a float from 0 to 1. If total_emissions cannot be found, return the final score as 0.5', 'final_score': ''}}
Extract the total carbon emissions in tcO2e for the year 2023. This may be presented as 'Total Scope 1 and 2 Emissions'. Please output the extracted values in the specified structure seen in the key-value pair for extracted_values below and try not to leave any blanks. Write

## Testing the Generator
Setting up LLM API 

In [17]:
import httpx
genai.configure(api_key=google_api_key)
llm_genai = genai.GenerativeModel('gemini-2.0-flash')

llm_openai = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
    http_client= httpx.Client()
)
#llm_deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")


reranker_model_name = "BAAI/bge-reranker-base"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
reranker_model = AutoModelForSequenceClassification.from_pretrained(reranker_model_name)
reranker_model.eval()  

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [33]:
def generate_genai_response(query, reranked_docs):

    """Retrieve context from ChromaDB and generate an answer using DeepSeek."""
    
    context = "\n\n".join(reranked_docs)

    prompt = f"""You are an expert in ESG analysis. Please reason through step by step and then provide the final answer to the query. 
    Please verify your answer against the context provided, and rewrite the answer if inconsistent. Below is a question and relevant retrieved documents.
    
    Question: {query}

    Context:
    {context + "End of Context"}

    Please provide a factually accurate response. If a fact is used from a document, include 'ids' next to it, like this. *information* ("doc_xxxxx")
    """
    retries = 3
    while retries > 0:
        try:
          completion = llm_genai.generate_content(prompt)

          if completion and completion.text:
                return {"text": completion.text.strip()}
          else:
                print("Empty completion received. Retrying...")
                retries -= 1
                time.sleep(2)
        except Exception as e:
            print(f"API Error encountered: {e}. Retrying after delay...")
            retries -= 1
            time.sleep(5)

    return "API Error: Unable to generate response after retries."

def generate_openai_response(query, reranked_docs):
    print("Chatgpttt")

    """Retrieve context from ChromaDB and generate an answer using DeepSeek."""
    
    context = "\n\n".join(reranked_docs)

    prompt = f"""You are an expert in ESG analysis. Please reason through step by step and then provide the final answer to the query. 
    Please verify your answer against the context provided, and rewrite the answer if inconsistent. Below is a question and relevant retrieved documents.
    
    Question: {query}

    Context:
    {context + "End of Context"}

    """
    retries = 3
    while retries > 0:
        try:
            response = llm_openai.chat.completions.create(
                model="gpt-3.5-turbo",  # or your preferred model
                messages=[
                    {"role": "assistant", "content": "You are an expert in ESG analysis looking through several documents"},
                    { "role": "user", "content": prompt},
                ],
            )
            if response and response.choices:
                return {"text": response.choices[0].message.content.strip()}
            else:
                print("Empty completion received. Retrying...")
                retries -= 1
                time.sleep(2)
        except Exception as e:
            print(f"API Error encountered: {e}. Retrying after delay...")
            retries -= 1
            time.sleep(5)
    return "API Error: Unable to generate response after retries."

'''def generate_deepseek_response(query, reranked_docs):
    print("deepseeking")

    """Retrieve context from ChromaDB and generate an answer using DeepSeek."""
    
    context = "\n\n".join(reranked_docs)

    prompt = f"""You are an expert in ESG analysis. Please reason through step by step and then provide the final answer to the query. 
    Please verify your answer against the context provided, and rewrite the answer if inconsistent. Below is a question and relevant retrieved documents.
    
    Question: {query}

    Context:
    {context + "End of Context"}

    Please provide a factually accurate response. If a fact is used from a document, include 'ids' next to it, like this. *information* ("doc_xxxxx")
    """
    retries = 3
    while retries > 0:
        try:
            response = llm_deepseek.completions.create(
                engine="deepseek/deepseek-r1-zero:free",  # or your preferred model
                messages=[
                    { "role": "user", "content": prompt},
                ],
                temperature=0.7,
            )
            if response['choices'][0] and response['choices'][0]['text']:
                return {"text": response['choices'][0]['text']}
            else:
                print("Empty completion received. Retrying...")
                retries -= 1
                time.sleep(2)
        except Exception as e:
            print(f"API Error encountered: {e}. Retrying after delay...")
            retries -= 1
            time.sleep(5)
    return "API Error: Unable to generate response after retries."'''

def rerank_documents(query, retrieved_docs):
    """
    Reranks the retrieved documents based on relevance scores using the BAAI/bge-reranker-base model.

    Args:
        query (str): The search query.
        retrieved_docs (list): A list of retrieved document texts.

    Returns:
        list: The reranked documents sorted by relevance.
    """
    if not retrieved_docs:
        return []

    # tokenise the query? = solve the ranking
    query_inputs = reranker_tokenizer(query, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        query_outputs = reranker_model(**query_inputs,output_hidden_states=True)
    # For simplicity, use the [CLS] token (first token) as the query embedding.
    query_embedding = query_outputs.hidden_states[-1][:, 0]
    
    doc_inputs = reranker_tokenizer(retrieved_docs, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        doc_outputs = reranker_model(**doc_inputs,output_hidden_states=True)
    # Use the [CLS] token embedding for each document.
    doc_embeddings = doc_outputs.hidden_states[-1][:, 0]

    # Compute relevance scores
    similarities = F.cosine_similarity(query_embedding, doc_embeddings, dim=-1)  # shape: (num_docs,)

    # Sort retrieved docs by relevance score (descending order)
    sorted_indices = similarities.argsort(descending=True)
    reranked_docs = [retrieved_docs[i] for i in sorted_indices.tolist()]
    return reranked_docs


# Start of full RAG code with json

In [35]:
def retrieve_company_metadata(company_tuple):
    """
    Query the collection to retrieve metadata for a given company.
    Expects company_tuple to be (company_name, year).
    """
    chroma_client = chromadb.PersistentClient(path="../chromadb_1003")
    collection = chroma_client.get_or_create_collection(name="dsa3101")
    results = collection.query(
        query_texts=[""],  # no text query; we use metadata filtering only
        n_results=1,
        where={"$and": [{"company": company_tuple[0]}, {"year": company_tuple[1]}]}
    )
    if not results.get("documents") or not results["documents"][0]:
        print(f"Company '{company_tuple[0]}' for year '{company_tuple[1]}' is not in the database. Exiting.")
        sys.exit(1)

    # Assumes metadata is stored along with the document. For example:
    # metadatas: [[{"company": "DBS", "year": 2023.0, "country": "Singapore", "industry": "Finance"}]]
    metadata = results["metadatas"][0][0]
    if "country" not in metadata or "industry" not in metadata:
        print(f"Metadata for {company_tuple[0]} is missing 'country' or 'industry' fields. Exiting.")
        sys.exit(1)
    return metadata

def retrieve_esg_text(company_tuple, query):
    """
    Retrieve ESG text from ChromaDB for a company given a specific query.
    """
    results = collection.query(
        query_texts=[query],
        n_results=15,
        where={"$and": [{"company": company_tuple[0]}, {"year": company_tuple[1]}]}
    )
    if not results.get("documents") or not results["documents"][0]:
        print(f"Company '{company_tuple[0]}' for year '{company_tuple[1]}' is not in the database. Exiting.")
        sys.exit(1)
    return results

def get_reranked_docs(query, results):
    retrieved_docs = [doc for doc in results["documents"][0]]
    reranked_docs = rerank_documents(query, retrieved_docs)
    return reranked_docs

def extract_values(query, results):
    reranked_docs = get_reranked_docs(query, results)
    response = generate_openai_response(query, reranked_docs)
    return response

def compute_linear_score(extracted_values, scoring_query):
    final_answer_generator = 'Return your answer as: "Final Answer: X" (where X is a numeric or clearly defined answer)'
    score = generate_openai_response(scoring_query + final_answer_generator, str(extracted_values))
    return score

def extract_absolute_score(score):
    matches = re.findall(r"(?:Final\s*Answer|Answer).*?([-+]?\d*\.?\d+)(?:\s*\%\.)?", str(score), re.DOTALL)
    return float(matches[-1]) if matches else "N/A"

def process_company(company_tuple):

    metadata = retrieve_company_metadata(company_tuple)
    country = metadata.get("country", "").strip().lower()
    industry = metadata.get("industry", "").strip().lower()
    
    # Choose JSON query file and CSV file based on country and industry.
    if country == "singapore":
        if industry == "finance":
            query_file = "../../files/scoring_queries/sg_bank_query.json"
            csv_file = "sg_finance_score.csv"
        elif industry == "health":
            query_file = "../../files/scoring_queries/sg_healthcare_query.json"
            csv_file = "sg_healthcare_score.csv"
        else:
            print(f"Unsupported industry '{industry}' for country '{country}'. Exiting.")
            sys.exit(1)
    else:
        print(f"Unsupported country '{country}'. Exiting.")
        sys.exit(1)

    # Load the ESG metrics from the JSON file
    try:
        with open(query_file, "r") as file:
            esg_metrics = json.load(file)
    except Exception as e:
        print(f"Error loading JSON file '{query_file}': {e}")
        sys.exit(1)
        
    # Prepare DataFrame headers based on the metrics in the JSON file.
    headers = [list(item.keys())[0] for item in esg_metrics]
    df_columns = ["Company", "Year"] + headers  # one column per ESG metric
    
    # Load existing CSV for this combination if it exists
    if os.path.exists(csv_file):
        df_existing = pd.read_csv(csv_file)
        existing_companies = set(map(tuple, df_existing[['Company', 'Year']].values.tolist()))
    else:
        df_existing = pd.DataFrame(columns=df_columns)
        existing_companies = set()
        
    # Skip if company has already been processed for this CSV
    if tuple(company_tuple) in existing_companies:
        print(f"Company '{company_tuple[0]}' for year '{company_tuple[1]}' already processed; skipping.")
        return None, csv_file, df_columns

    row_data = {"Company": company_tuple[0], "Year": company_tuple[1]}
    
    # Process each ESG metric defined in the JSON file.
    for metric_item in esg_metrics:
        for metric, details in metric_item.items():
            metric_name = metric
            query = details["value_query"]
            scoring_thresholds = details["scoring_query"]

            # Retrieve ESG text using the query.
            retrieved_text = retrieve_esg_text(company_tuple, query)
            # Extract values using DeepSeek (or your equivalent method).
            extracted_values = extract_values(query, retrieved_text)
            # Compute the linear score.
            score = compute_linear_score(extracted_values, scoring_thresholds)
            
            row_data[metric_name] = {"extracted_values": extracted_values, "score": score}
    
    return row_data, csv_file, df_columns




# Use a dictionary to group new rows by the target CSV file.
def extract_esgreports(companies):
    

    new_rows = {}

    for company_tuple in companies:
        # process_company is assumed to be defined elsewhere
        row_data, csv_file, df_columns = process_company(company_tuple)

        # If no row_data, skip
        if row_data is None:
            continue

        # If this CSV hasn't been seen yet, initialize a new entry
        if csv_file not in new_rows:
            new_rows[csv_file] = {"rows": [], "columns": df_columns}

        # Add the row data to our in-memory collection
        new_rows[csv_file]["rows"].append(row_data)

    # Once all rows are collected, write or append them to their respective CSVs
    for csv_file, data in new_rows.items():
        df_new = pd.DataFrame(data["rows"], columns=data["columns"])

        if os.path.exists(csv_file):
            df_existing = pd.read_csv(csv_file)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True)
        else:
            df_combined = df_new

        df_combined.to_csv(csv_file, index=False)
        print(f"Added {len(data['rows'])} new companies to {csv_file}.")


In [36]:
# List of companies to process (each as a tuple: (Company, Year))
companies = [
    ("DBS", 2023.0),
    ("UOBGROUP", 2023.0),
    ("OCBC", 2023.0),
    ("ECONHEALTHCARE", 2024.0),
    ("FULLERTONHEALTH", 2023.0),
    ("HEALTHMEDICAL", 2022.0),
    ("HSA", 2023.0)
]

extract_esgreports(companies)


Company 'DBS' for year '2023.0' already processed; skipping.
Company 'UOBGROUP' for year '2023.0' already processed; skipping.
Company 'OCBC' for year '2023.0' already processed; skipping.
Company 'ECONHEALTHCARE' for year '2024.0' already processed; skipping.
Company 'FULLERTONHEALTH' for year '2023.0' already processed; skipping.
Company 'HEALTHMEDICAL' for year '2022.0' already processed; skipping.
Company 'HSA' for year '2023.0' already processed; skipping.


### SubIndusry Score database
Extract the scores from the sub_industry csv

1. Parse the extracted values to get final answer, then return final score with metric into json saved as a subindustry


In [9]:
def extract_to_json(subindustry_filepath):
    csv_file = subindustry_filepath

    df = pd.read_csv(csv_file)

    company_col = df.columns[0]
    year_col = df.columns[1]
    other_cols = df.columns[2:]  

    data_dict = {}

    for _, row in df.iterrows():
        company = row[company_col]
        year = row[year_col]
        details = {}
    
        for col in other_cols:
            value = row[col]
            extracted = extract_absolute_score(value)
            details[col] = extracted if extracted is not None else value

        if company not in data_dict:
            data_dict[company] = {}
            data_dict[company][year] = details

    json_file = os.path.splitext(csv_file)[0] + ".json"

    with open(json_file, "w") as f:
      json.dump(data_dict, f, indent=4)

    print("Conversion complete. JSON file saved as:", json_file)

extract_to_json('sg_healthcare_score.csv')
extract_to_json('sg_finance_score.csv')

Conversion complete. JSON file saved as: sg_healthcare_score.json
Conversion complete. JSON file saved as: sg_finance_score.json


### Vector database
When you store documents in ChromaDB using collection.add(), it:

1. Generates vector embeddings for your text (if you haven't provided them).
2. Stores the document along with its embedding in the vector database.
3. Matches queries based on similarity search (cosine similarity by default).


# Post processing

Checking for hallucination, irrelevance, bias 
In this assignment, I felt that biasness wasn't really a metric required, I think it would be good to add biasness if i extracted data from third party sources grading the company esg scores. I can then compare the third-party metrics and scoring to each company's esg reports, and check if there is biasness in terms of their ratings, towards a particular, company or industry, etc. Therefore, I just added the metric for future reference, but it is not required in this assignment.

### Hallucination detection (Faithfullness)

In [ ]:
def normalize_text(text):
    """Normalize text by converting to lowercase and removing punctuation."""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

def fuzzy_match(sentence, doc, threshold=80):
    """Check if sentence has a fuzzy match in the document."""
    return fuzz.partial_ratio(normalize_text(sentence), normalize_text(doc)) >= threshold

def verify_facts(response, reranked_docs, fuzzy_threshold=80):
    """Detect hallucinations by checking if sentences exist in retrieved docs using fuzzy matching."""
    missing_facts = []
    
    # Split response into sentences and check if they appear in any of the documents
    for sent in response.split(". "):
        found = any(fuzzy_match(sent, doc, fuzzy_threshold) for doc in reranked_docs)
        if not found:
            missing_facts.append(sent)

    if missing_facts:
        print("Warning: Some statements are not found in the retrieved context:")
        for fact in missing_facts:
            print(f"- {fact}")
    
    return 1 - len(missing_facts) / len(response.split(". "))  # Faithfulness Score

faithfulness_score = verify_facts(response, reranked_documents)
print(f"Faithfulness Score: {faithfulness_score}")

## Irrelevance Check

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def check_relevance(query, response, threshold=0.6):
    """
    Check the relevance of the response to the query using semantic similarity.
    """
    query_embedding = model.encode([query])
    response_embedding = model.encode([response])

    similarity = cosine_similarity(query_embedding, response_embedding)[0][0]

    if similarity >= threshold:
        return True, similarity  
    else:
        return False, similarity  

In [ ]:
is_relevant = check_relevance(query, response)
print(f"Is the response relevant? {is_relevant}")

In [ ]:
model_name = "BAAI/bge-reranker-base" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

def check_bias(text):
    """
    Check for potential bias in the text using a pretrained model.
    """
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        logits = model(**inputs).logits

    # Assuming binary classification (0 = no bias, 1 = biased)
    predicted_class = torch.argmax(logits, dim=1).item()
    
    return predicted_class == 1  # 1 indicates bias (this depends on the model's labeling)

# Example Usage
response = "Pfizer has been focusing on improving diversity in their clinical trials and sharing their insights with others as part of their diversity and inclusion initiatives in 2022."

is_biased = check_bias(response)
print(f"Is the response biased? {is_biased}")

# Evaluation

## Retriever Evaluation
Typical metrics: RecalL@k, Precision @k, Mean Reciprocal Rank, Mean Average Precision

### Cosine similarity

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_retrieval_relevance(query, reranked_docs):
    """Compute semantic similarity between query and retrieved docs."""
    corpus = [query] + reranked_docs
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    similarity_scores = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:])
    return similarity_scores.mean()  # Average similarity

In [ ]:
compute_retrieval_relevance(query, reranked_documents)

## Generator Evaluation 
Typical metrics: ROUGE, BLEU, BERTScore, domain-specific or task-specific metrics

### BLEU Score (Text similarity)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def compute_bleu_score(reference, generated_response):
    """Compare generated response against reference text using BLEU score."""
    reference_tokens = reference.lower().split()
    generated_tokens = generated_response.lower().split()
    return sentence_bleu([reference_tokens], generated_tokens)

In [ ]:
compute_bleu_score(query, response)

### Retrieval score (relevance)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def compute_retrieval_relevance(reranked_docs, response):
    """Calculate how relevant the response is to the retrieved documents."""
    corpus = reranked_docs + [response]  # Combine all docs and response
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    similarity_matrix = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    return similarity_matrix.mean()  # Average similarity score


In [ ]:
compute_retrieval_relevance(reranked_docs, response)

### Judge LM 

In [ ]:
client = genai.Client(api_key=google_api_key)
reranked_docs_str = "\n".join(reranked_documents)

gemini_eval = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=f"""
                Evaluate how well the response answers the query, giving an explanation of how it answers the question, and whether the response is factually correct based on the context provided.
                I have added the query, response and retrieved context below.
                
                Query: 
                {query}
                
                Response:
                {response}
                
                Retrieved Context:
                {reranked_docs_str}
                
                Give a score from 0 to 10, and a detailed explanation on the score, where:
                - 10 = Perfectly accurate
                - 0 = Completely incorrect
                """
)

print(gemini_eval.text)